In [2]:
import weaviate

# connect to an already-running local Weaviate instance
client = weaviate.connect_to_local(port=8079, grpc_port=50050)

# Quick check: list collections (matches patterns used in the notebooks)
print("Collections:", list(client.collections.list_all().keys()))

Collections: ['Bbc_collection']


In [3]:
import time
import weaviate

def connect_local_with_retry(port=8079, grpc_port=50050, retries=5, delay=2.0):
    last_exc = None
    for i in range(retries):
        try:
            client = weaviate.connect_to_local(port=port, grpc_port=grpc_port)
            # Some clients have convenience health/is_ready helpers; if not, try a light API call:
            # attempt to list collections — if not possible an exception will be raised
            _ = client.collections.list_all()
            return client
        except Exception as e:
            last_exc = e
            time.sleep(delay)
    # If we get here, all retries failed
    raise RuntimeError(f"Could not connect to local Weaviate after {retries} attempts") from last_exc

client = connect_local_with_retry()
print("Connected to Weaviate, collections:", list(client.collections.list_all().keys()))

Connected to Weaviate, collections: ['Bbc_collection']


/opt/conda/lib/python3.11/site-packages/weaviate/warnings.py:302: ResourceWarning: Con004: The connection to Weaviate was not closed properly. This can lead to memory leaks.
            Please make sure to close the connection using `client.close()`.
  warnings.warn(
/tmp/ipykernel_780/88772548.py:19: ResourceWarning: unclosed <socket.socket fd=70, family=2, type=1, proto=6, laddr=('127.0.0.1', 34466), raddr=('127.0.0.1', 8079)>
  client = connect_local_with_retry()


In [4]:
import os

os.environ["COLLECTIONS_PATH"] = "./collections"

In [ ]:
# in weaviate_server.py (already present in your repo)
client = weaviate.connect_to_embedded(persistence_data_path=os.environ["COLLECTIONS_PATH"], ...)

In [ ]:
# create a collection only if it doesn't exist
collection_name = "example_collection"
if not client.collections.exists(collection_name):
    # Example collection schema / config
    client.collections.create(
        name=collection_name,
        vectorizer="text2vec-transformers",  # example, use what's available in your server
        vector_index_type="hnsw",            # example options — follow your server modules
        vector_index_config={"ef": 128}
    )
collection = client.collections.get(collection_name)

# insert a single object (properties match what your repo uses, e.g. 'title','chunk','link','pubDate')
properties = {
    "title": "Example",
    "chunk": "This is a sample chunk",
    "link": "https://example.org",
    "pubDate": "2024-01-01"
}
# Insert individual object
collection.data.insert(properties=properties)

In [6]:
collection = client.collections.get("bbc_collection")

In [8]:
from weaviate.classes.query import Rerank, Filter

# semantic (near text) search
results = collection.query.near_text(limit=3, query="Tell me about Taylor Swift")
# convert to list of property dicts (same pattern as your notebooks)
hits = [obj.properties for obj in results.objects]
for h in hits:
    print(h['title'], "->", h['chunk'][:120])

WeaviateQueryError: Query call with protocol GRPC search failed with message explorer: get class: vectorize params: could not vectorize input for collection Bbc_collection with search-type nearText. Make sure a vectorizer module is configured for this collection.

In [ ]:
# https://docs.weaviate.io/weaviate/quickstart/local

